# LinkedIn Web Scraping using Selenium 

In [108]:
# importing webdriver will help in creating a dummy web browser for us.
from selenium import webdriver

# we use this library to ensure there is a span of time between requests.
import time

# pandas library helps in generating to DataFrames.
import pandas as pd


# import selenium By to locate to elements to scraped 
from selenium.webdriver.common.by import By

import requests


 We will be using a Google Chrome driver. We can download it from this site:  https://chromedriver.chromium.org/downloads

### Move the driver file to a PATH
Go to the Downloads directory, unzip the file and move it to 'C:/chromedrivers/chromedriver.exe' PATH.

In [109]:
driver = webdriver.Chrome(r'C:/chromedrivers/chromedriver.exe')


<ipython-input-109-813da720f857>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:/chromedrivers/chromedriver.exe')


In [110]:
#input URL
driver.get('https://www.linkedin.com/')
time.sleep(3)


In [111]:
# For Login into LinkedIn
username = driver.find_element(by=By.ID, value="session_key")
username.send_keys("mohammedsyedtaher1@gmail.com")#provide linkedin login user_id in the blank. 
password = driver.find_element(by=By.ID, value="session_password")
password.send_keys("Msdt@8328") #provide linkedin login password in the blank.
time.sleep(2)

![LinkedIn](https://imgur.com/hYqdJHc.png)

In [112]:
# Submit Button click
login_button = driver.find_element(by=By.CLASS_NAME, value="sign-in-form__submit-button")
login_button.click()
time.sleep(2)

In [113]:
# Input URL for Linkedin jobs
driver.get("https://www.linkedin.com/jobs/search/?keywords=web%20developer")
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(5)

# Job Category:Information Technology


In [114]:
Sub_Category =['Web Developer']

In [115]:
# Input URL for Linkedin jobs

for job in Sub_Category:
    driver.get("https://www.linkedin.com/jobs/search/?keywords=" + job)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

In [116]:
# Creating empty Python Dictionary
dictn = {'job_title': [], 'company': [], 'location': []}
titles = driver.find_elements(by=By.XPATH, value="//div[@class='full-width artdeco-entity-lockup__title ember-view']/a")
company = driver.find_elements(by=By.XPATH, value="//div[@class='artdeco-entity-lockup__subtitle ember-view']/a")
location = driver.find_elements(by=By.XPATH, value='//div[@class="artdeco-entity-lockup__caption ember-view"]/ul/li[1]')

In [117]:
# Store data into Python Dictionary
index = min(len(titles),len(company),len(location))
print(index)
for i in range(index):
    dictn['job_title'].append(titles[i].text)
    dictn['company'].append(company[i].text)
    dictn['location'].append(location[i].text)

7


In [118]:
# Creating CSV file using Pandas 
df = pd.DataFrame(dictn)
df


,job_title,company,location
0,PHP Web Developer,ZeetaPro Inc.,"Kolkata, West Bengal, India"
1,Frontend Developer,SkillVertex,"Bengaluru, Karnataka, India"
2,Frontend Developer,Reed & Willow Talent Advisory Group,"Pune, Maharashtra, India"
3,Application Developer- Full Stack Developer,YTSolutions (I) Pvt Ltd,"Bengaluru, Karnataka, India"
4,Frontend Developer,EdSanta Education,India
5,Web Developer,TypeTail,"Chennai, Tamil Nadu, India"
6,Web Developer,Apna Godam,"Jaipur, Rajasthan, India"


# Creating Database Table for Jobs

In [138]:
import sqlite3

#create connection
conn = sqlite3.connect('jobs.db')
c = conn.cursor()

In [139]:
#create a table
c.execute('''CREATE TABLE job(job_title  TEXT,company TEXT,location TEXT)''')

OperationalError: table job already exists

In [140]:
for i in range(6):

    c.execute('''INSERT INTO job VALUES(?,?,?)''', ((dictn["job_title"][i]), (dictn["company"][i]), (dictn["location"][i])))
conn.commit()


In [141]:
#select all data from table and print
c.execute('''SELECT * FROM job''')
results = c.fetchall()
print(results)

[('Full Stack Developer - Intern', 'Xdigics Technologies Private Limited', 'Bengaluru, Karnataka, India'), ('Web Developer', 'Prishth Technosoft', 'India'), ('Full Stack Engineer', 'Job In Gujarat', 'Gandhinagar, Gujarat, India'), ('Web Developer', "Pethanatchi's Studio of Designing", 'Sivakasi, Tamil Nadu, India'), ('Full Stack Engineer', 'IMS Group', 'Jaipur, Rajasthan, India'), ('Junior Web Developer', 'WTW', 'Ahmedabad, Gujarat, India'), ('PHP Web Developer', 'ZeetaPro Inc.', 'Kolkata, West Bengal, India'), ('Frontend Developer', 'SkillVertex', 'Bengaluru, Karnataka, India'), ('Frontend Developer', 'Reed & Willow Talent Advisory Group', 'Pune, Maharashtra, India'), ('Application Developer- Full Stack Developer', 'YTSolutions (I) Pvt Ltd', 'Bengaluru, Karnataka, India'), ('Frontend Developer', 'EdSanta Education', 'India'), ('Web Developer', 'TypeTail', 'Chennai, Tamil Nadu, India'), ('PHP Web Developer', 'ZeetaPro Inc.', 'Kolkata, West Bengal, India'), ('Frontend Developer', 'Skill

In [123]:

# Storing URL of Same Companies to get Data
links = []
for a in driver.find_elements(by=By.XPATH, value='//div[@class="mr1 artdeco-entity-lockup__image artdeco-entity-lockup__image--type-square ember-view"]/a'):
    # print(a.get_attribute('href'))
    links.append(a.get_attribute('href'))



dict1 = {"description": [], "location":[], "no._of_employee": []}
for link in links:
    source = requests.get(link)

    try:
        driver.get(link)
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, 900)")
        time.sleep(2)
        
        dict1["description"].append(driver.find_element(by=By.XPATH, value='//div[@class="jobs-company__box"]/p/div').text)
        dict1["no._of_employee"].append(driver.find_element(by=By.XPATH, value='//span[1][@class="jobs-company__inline-information"]').text)
        dict1["location"].append(driver.find_element(by=By.CLASS_NAME, value='jobs-unified-top-card__bullet').text)
    except:
        print("URL not found")

URL not found


In [125]:
# Creating CSV file using Pandas 
df1 = pd.DataFrame(dict1)
df1

,description,location,no._of_employee
0,ZeetaPro Inc. (Creative Mileage Solutions Pvt....,"Kolkata, West Bengal, India",11-50 employees
1,An e-learning platform with a vision to upskil...,"Bengaluru, Karnataka, India",201-500 employees
2,Reed & Willow- The Story\nOurs is a generation...,"Pune, Maharashtra, India",11-50 employees
3,YTSolutions - an LA based technology consultin...,"Bengaluru, Karnataka, India",11-50 employees
4,We are an innovative education technology prod...,India,51-200 employees
5,TypeTail is a Augmented Reality studio buildin...,"Chennai, Tamil Nadu, India",2-10 employees


# Data Base for Company Details

Company Details: - Name, Description, State (States), Subcategory (Job types 2).


In [151]:
#create connection
conn = sqlite3.connect('Company_details1.db')
c = conn.cursor()

In [152]:
#create a table
c.execute('''CREATE TABLE Company_details1(Name TEXT,Description TEXT,State TEXT,Subcategory TEXT)''')

In [153]:
for i in range(5):
    c.execute('''INSERT INTO Company_details1 VALUES(?,?,?,?)''',((dictn["company"][i]), dict1["description"][i],(dictn["location"][i]),(dictn["job_title"][i]) ))

In [154]:
#select all data from table and print
c.execute('''SELECT * FROM Company_details1''')
results = c.fetchall()
print(results)

[('ZeetaPro Inc.', 'ZeetaPro Inc. (Creative Mileage Solutions Pvt. Ltd.) have had a long history of developing custom web applications for its US based clients ranging from eCommerce, Online Reservations and Registrations, Content Management Systems, Media Library, Document Management, Product Information Management and other applications for smarter business processing. After development and deployment of solutions, we continued to provide technical support to our loyal customer base.\n\nAs part of its evolution and new strategic focus, the Company now offers its own proprietary SaaS solutions in Business Processes Automation to enable higher efficiency and easier administration.\nThe Kolkata based Indian subsidiary is called Creative Mileage Solutions Pvt. Ltd.\n…\nshow more', 'Kolkata, West Bengal, India', 'PHP Web Developer'), ('SkillVertex', 'An e-learning platform with a vision to upskill students for the industry and help you land your dream Job/University.', 'Bengaluru, Karnata

# Future work

Using for loops and lists, multiple job sub categories can be scraped for a given category in a given state

In [ ]:
jobs-unified-top-card__company-name